## Run this notebook

You can launch this notebook in the US GHG Center JupyterHub by clicking the link below.

[Launch in the US GHG Center JupyterHub (requires access)](https://hub.ghg.center/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2FUS-GHG-Center%2Fghgc-docs&urlpath=lab%2Ftree%2Fghgc-docs%2Fuser_data_notebooks%2Fgosat-based-ch4budget-yeargrid-v1_User_Notebook.ipynb&branch=main)
   

## Approach

1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the gridded methane emissions data product.
2. Pass the STAC item into the raster API `/collections/{collection_id}/items/{item_id}/tilejson.json `endpoint.
3. Using `folium.plugins.DualMap`, we will visualize two tiles (side-by-side), allowing us to compare time points. 
4. After the visualization, we will perform zonal statistics for a given polygon.

   

## About the Data

The NASA Carbon Monitoring System Flux (CMS-Flux) team analyzed remote sensing observations from Japan’s Greenhouse gases Observing SATellite (GOSAT) to produce the global Committee on Earth Observation Satellites (CEOS) CH₄ Emissions data product. They used an analytic Bayesian inversion approach and the GEOS-Chem global chemistry transport model to quantify annual methane (CH₄) emissions and their uncertainties at a spatial resolution of 1° by 1° and then projected these to each country for 2019.

For more information regarding this dataset, please visit the [GOSAT-based Top-down Total and Natural Methane Emissions](https://earth.gov/ghgcenter/data-catalog/gosat-based-ch4budget-yeargrid-v1) data overview page.

# Install the Required Libraries
Required libraries are pre-installed on the GHG Center Hub. If you need to run this notebook elsewhere, please install them with this line in a code cell:

%pip install requests folium rasterstats pystac_client pandas matplotlib --quiet

## Querying the STAC API
First, we are going to import the required libraries. Once imported, they allow better executing a query in the GHG Center Spatio Temporal Asset Catalog (STAC) Application Programming Interface (API) where the granules for this collection are stored.

In [1]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# Please use the collection name similar to the one used in STAC collection.

# Name of the collection for gosat budget methane. 
collection_name = "gosat-based-ch4budget-yeargrid-v1"

In [ ]:
# Fetching the collection from STAC collections using appropriate endpoint.
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

Examining the contents of our `collection` under the `temporal` variable, we see that the data is available from January 2010 to December 2022. By looking at the `dashboard:time density`, we observe that the data is available for thirteen years.

In [8]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [9]:
# Check total number of items available
number_of_items = get_item_count(collection_name)
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]
print(f"Found {len(items)} items")

Found 13 items


In [13]:
# Examining the first item in the collection
items[0]

{'id': 'gosat-based-ch4budget-yeargrid-v1-test-2022',
 'bbox': [-182.5, -90.9777777777778, 177.5, 90.97777777777777],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/api/stac/collections/gosat-based-ch4budget-yeargrid-v1-test'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/api/stac/collections/gosat-based-ch4budget-yeargrid-v1-test'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://dev.ghg.center/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://dev.ghg.center/api/stac/collections/gosat-based-ch4budget-yeargrid-v1-test/items/gosat-based-ch4budget-yeargrid-v1-test-2022'},
  {'title': 'Map of Item',
   'href': 'https://dev.ghg.center/api/raster/collections/gosat-based-ch4budget-yeargrid-v1-test/items/gosat-based-ch4budget-yeargrid-v1-test-2022/map?assets=post-total&rescale=0%2C0.3&colormap_name=spectral_r',
   'rel': 'pre

Below, we enter minimum and maximum values to provide our upper and lower bounds in `rescale_values.`

## Exploring Changes in GOSAT Methane budgets (CH4) Levels Using the Raster API

In this notebook, we will explore the impacts of methane emissions and by examining changes over time in urban regions. We will visualize the outputs on a map using `folium.`

In [14]:
# To access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["properties"]["datetime"][:10]: item for item in items} 
asset_name = "post-total"

In [15]:
# Fetching the min and max values for a specific item
rescale_values = {"max":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

In [ ]:
items.keys()

Now, we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this for first January 2019.

In [ ]:
color_map = "rainbow" # please select the color ramp from matplotlib library.
january_2019_tile = requests.get(
    f"{RASTER_API_URL}/collections/{items['2019-01-01']['collection']}/items/{items['2019-01-01']['id']}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
january_2019_tile

## Visualizing CH₄ Emissions


In [20]:
# Set initial zoom and center of map for CH₄ Layer
# Centre of map [latitude,longitude]
map_ = folium.Map(location=(34, -118), zoom_start=2)

# January 2019
map_layer_2019 = TileLayer(
    tiles=january_2019_tile["tiles"][0],
    attr="GHG",
    opacity=0.7,
)
map_layer_2019.add_to(map_)

# # January 2012
# map_layer_2012 = TileLayer(
#     tiles=january_2012_tile["tiles"][0],
#     attr="GHG",
#     opacity=0.7,
# )
# map_layer_2012.add_to(map_.m2)

# visualising the map
map_

# Calculating Zonal Statistics

To perform zonal statistics, first we need to create a polygon. In this use case we are creating a polygon in Texas (USA).

In [21]:
# Texas, USA
texas_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                # [13.686159004559698, -21.700046934333145],
                # [13.686159004559698, -23.241974326585833],
                # [14.753560168039911, -23.241974326585833],
                # [14.753560168039911, -21.700046934333145],
                # [13.686159004559698, -21.700046934333145],
                [-95, 29],
                [-95, 33],
                [-104, 33],
                [-104,29],
                [-95, 29]
            ]
        ],
        "type": "Polygon",
    },
}

In [22]:
# We will plug in the coordinates for a location inside the the polygon and a zoom level

aoi_map = Map(
    tiles="OpenStreetMap",
    location=[
        30,-100
    ],
    zoom_start=6,
)

folium.GeoJson(texas_aoi, name="Texas, USA").add_to(aoi_map)
aoi_map

In [23]:
# Check total number of items available
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit=300"
).json()["features"]
print(f"Found {len(items)} items")

Found 13 items


In [ ]:
# Explore the first item
items[0]

In [27]:
# The bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
def generate_stats(item, geojson):
    result = requests.post(
        f"{RASTER_API_URL}/cog/statistics",
        params={"url": item["assets"][asset_name]["href"]},
        json=geojson,
    ).json()
    return {
        **result["properties"],
        "datetime": item["properties"]["datetime"][:10],
    }

With the function above we can generate the statistics for the AOI.

In [28]:
%%time
stats = [generate_stats(item, texas_aoi) for item in items]

CPU times: user 160 ms, sys: 22.7 ms, total: 182 ms
Wall time: 7.4 s


In [29]:
stats[0]

{'statistics': {'b1': {'min': 0.45840875428297767,
   'max': 2.563434123331244,
   'mean': 1.3501146985384829,
   'count': 1.9799998998641968,
   'sum': 2.6732269679113765,
   'std': 0.5669339642223294,
   'median': 1.0878720382884925,
   'majority': 0.45840875428297767,
   'minority': 0.45840875428297767,
   'unique': 6.0,
   'histogram': [[2.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0],
    [0.45840875428297767,
     0.6689112911878043,
     0.8794138280926309,
     1.0899163649974577,
     1.300418901902284,
     1.5109214388071108,
     1.7214239757119376,
     1.931926512616764,
     2.1424290495215907,
     2.352931586426417,
     2.563434123331244]],
   'valid_percent': 100.0,
   'masked_pixels': 0.0,
   'valid_pixels': 6.0,
   'percentile_2': 0.45840875428297767,
   'percentile_98': 2.563434123331244}},
 'datetime': '2022-01-01'}

In [30]:
def clean_stats(stats_json) -> pd.DataFrame:
    df = pd.json_normalize(stats_json)
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]
    df["date"] = pd.to_datetime(df["datetime"])
    return df


df = clean_stats(stats)
df.head(5)

,datetime,min,max,mean,count,sum,std,median,majority,minority,unique,histogram,valid_percent,masked_pixels,valid_pixels,percentile_2,percentile_98,date
0,2022-01-01,0.458409,2.563434,1.350115,1.98,2.673227,0.566934,1.087872,0.458409,0.458409,6.0,"[[2.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0,...",100.0,0.0,6.0,0.458409,2.563434,2022-01-01
1,2021-01-01,0.417406,2.977322,1.409865,1.98,2.791533,0.719124,1.029306,0.417406,0.417406,6.0,"[[2.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,...",100.0,0.0,6.0,0.417406,2.977322,2021-01-01
2,2020-01-01,0.392606,2.619608,1.301333,1.98,2.576640,0.623242,0.978943,0.392606,0.392606,6.0,"[[2.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,...",100.0,0.0,6.0,0.392606,2.619608,2020-01-01
3,2019-01-01,0.521856,2.678034,1.409689,1.98,2.791184,0.667938,0.953206,0.521856,0.521856,6.0,"[[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0,...",100.0,0.0,6.0,0.521856,2.678034,2019-01-01
4,2018-01-01,0.552891,2.624957,1.427609,1.98,2.826666,0.582425,1.086807,0.552891,0.552891,6.0,"[[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0,...",100.0,0.0,6.0,0.552891,2.624957,2018-01-01


In [31]:
print(items[0]["properties"]["datetime"][:10])

2022-01-01


In [32]:
tile_2016 = requests.get(
    f"{RASTER_API_URL}/collections/{items[0]['collection']}/items/{items[0]['id']}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}",
).json()
tile_2016

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://dev.ghg.center/api/raster/collections/gosat-based-ch4budget-yeargrid-v1-test/items/gosat-based-ch4budget-yeargrid-v1-test-2022/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=post-total&color_formula=gamma+r+1.05&colormap_name=rainbow&rescale=-4.4208666115704%2C8.355603568623742'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-182.5, -90.9777777777778, 177.5, 90.97777777777777],
 'center': [-2.5, -1.4210854715202004e-14, 0]}

In [34]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds

aoi_map_bbox = Map(
    tiles="OpenStreetMap",
    location=[
        30,-100
    ],
    zoom_start=2,
)

map_layer = TileLayer(
    tiles=tile_2016["tiles"][0],
    attr="GHG", opacity = 0.5
)

map_layer.add_to(aoi_map_bbox)

aoi_map_bbox

## Summary

In this notebook we have successfully completed the following steps for the STAC collection for the GOSAT-based Top-down Total and Natural Methane Emissions dataset.

1.  Install and import the necessary libraries
2.  Fetch the collection from STAC collections using the appropriate endpoints
3.  Count the number of existing granules within the collection
4.  Map the methane emission levels
5.  Generate zonal statistics for the area of interest (AOI)

If you have any questions regarding this user notebook, please contact us using the [feedback form](https://docs.google.com/forms/d/e/1FAIpQLSeVWCrnca08Gt_qoWYjTo6gnj1BEGL4NCUC9VEiQnXA02gzVQ/viewform).